# How to deal with a Nuclear Shell Model (NSM) Hamiltonian -a funny guide to madness-

NSM Hamiltonians are many-body Hamiltonians represented in a single particle framework (or nucleon modes) described by Valence orbitals (with the famous set of quantum numbers that describe the 3D harmonic oscillator $|n,l,j,m,t,t_z\rangle$. The last two quantum numbers describe the isospin of the nucleons (proton or neutron).

First of all, we need to commit: we need to decide which Valence shell we are going to use. This choice translates in selecting the proper "nuclear_interaction.txt" file related to the correspoding valence shell.

- p shell --> 'data/cki' file 
- sd shell --> 'data/usdb.nat' file 
- pf shell --> 'data/gxpf1a' file 

Don't worry about these names, for you these simply are file text with matrix entries that are going to be the parameters of your NSM Hamiltonian:
$$
H=\sum_a c^{+}_a c_a + \frac{1}{4}\sum_{abcd} v_{abdc} c^{+}_a c^{+}_b c_c c_d.
$$

Ok, too much verbose, let's start with the Imports

In [1]:
from src.NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian # the many-body Hamiltonian class
from src.NSMFermions.nuclear_physics_utils import SingleParticleState,J2operator,get_twobody_nuclearshell_model # routines and class useful for the nuclear part
import scipy # just scipy, easy, no?
import numpy as np 
import matplotlib.pyplot as plt # to plot things

We initialize the Single Particle State class with the corresponding file text. We also initialize the number of proton neutron and the corresponding number of modes for each nucleon

In [2]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)
# single particle energies
print('single particle energies=',SPS.energies,'\n')
print('mapping between nucleon modes a and the quantum numbers',SPS.state_encoding)

nucleon_modes_per_isospin=SPS.energies.shape[0]//2 # we are counting per species. I know, we need a .num_modes attribute

num_neutrons=2
num_protons=2

labels= ['1', '3', '205', '1001', '203']
single particle energies= [-3.9257 -3.9257 -3.9257 -3.9257 -3.9257 -3.9257 -3.2079 -3.2079  2.1117
  2.1117  2.1117  2.1117 -3.9257 -3.9257 -3.9257 -3.9257 -3.9257 -3.9257
 -3.2079 -3.2079  2.1117  2.1117  2.1117  2.1117] 

mapping between nucleon modes a and the quantum numbers [(0, 2, 2.5, np.float64(-2.5), 0.5, 0.5), (0, 2, 2.5, np.float64(-1.5), 0.5, 0.5), (0, 2, 2.5, np.float64(-0.5), 0.5, 0.5), (0, 2, 2.5, np.float64(0.5), 0.5, 0.5), (0, 2, 2.5, np.float64(1.5), 0.5, 0.5), (0, 2, 2.5, np.float64(2.5), 0.5, 0.5), (1, 0, 0.5, np.float64(-0.5), 0.5, 0.5), (1, 0, 0.5, np.float64(0.5), 0.5, 0.5), (0, 2, 1.5, np.float64(-1.5), 0.5, 0.5), (0, 2, 1.5, np.float64(-0.5), 0.5, 0.5), (0, 2, 1.5, np.float64(0.5), 0.5, 0.5), (0, 2, 1.5, np.float64(1.5), 0.5, 0.5), (0, 2, 2.5, np.float64(-2.5), 0.5, -0.5), (0, 2, 2.5, np.float64(-1.5), 0.5, -0.5), (0, 2, 2.5, np.float64(-0.5), 0.5, -0.5), (0, 2, 2.5, np.float64(0.5), 0.5, -0.5), (0, 2, 2.5, np.float64(1.

Now, we can initialize the NSM Hamiltonian and look at the many-body basis. It is a tensor with the first index as the many-body basis index and second index the corresponding nucleon modes in the tensor product basis state

In [3]:
NSMHamiltonian=FermiHubbardHamiltonian(size_a=nucleon_modes_per_isospin,size_b=nucleon_modes_per_isospin,nparticles_a=num_neutrons,nparticles_b=num_protons,symmetries=[SPS.total_M_zero]) # the symmetry that we need is the M=0 condition, if we do not add anything we get the full many-body basis

print(NSMHamiltonian.basis)


[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 1]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


Now we initialize the NSM Hamiltonian, starting from the external potential $\sum_a e_a c^{+}_a c_a$

In [4]:
NSMHamiltonian.get_external_potential(external_potential=SPS.energies)

Then we go to the two-body term, loading the matrix dict

In [5]:
print(SPS.state_encoding[19],SPS.state_encoding[2],SPS.state_encoding[1],SPS.state_encoding[16])



(1, 0, 0.5, np.float64(0.5), 0.5, -0.5) (0, 2, 2.5, np.float64(-0.5), 0.5, 0.5) (0, 2, 2.5, np.float64(-1.5), 0.5, 0.5) (0, 2, 2.5, np.float64(1.5), 0.5, -0.5)


In [6]:
twobody_dict,_=get_twobody_nuclearshell_model(file_name=file_name)

NSMHamiltonian.get_twobody_interaction(twobody_dict)



labels= ['1', '3', '205', '1001', '203']
Computing the matrix, pls wait... (u_u) 



100%|██████████| 24/24 [00:54<00:00,  2.26s/it]
16496it [01:44, 158.14it/s]


In order to use the hamiltonian, we need to compute the full Hamiltonian matrix, using get_hamiltonian()

In [7]:
NSMHamiltonian.get_hamiltonian()

At this point, getting the spectrum is easy as getting the empadronamiento in Barcelona (joke)

In [8]:
egs,psigs=NSMHamiltonian.get_spectrum(n_states=1) #we are only interested in the gs 

print(f'energy ground state={egs[0]:.5} Mev \n')


energy ground state=-41.396 Mev 



We can also play with other attributes that the Hamiltonian can provide, such as 2-body operators $T_{ab}^{cd}=c^{+}_a c^{+}_{b} c_c c_d$

In [20]:
t_0312=NSMHamiltonian.adag_adag_a_a_matrix(0,3,1,2) # it's a matrix in the many-body basis

# we compute the expectation value using Scipy <psi| T_01^23 |psi>
expectation_value=psigs[:,0].conjugate().dot(t_0312.dot(psigs[:,0])) # the 0 index is because  psigs \in [dim(Hilbert space),n_states]
print('expectation value=',expectation_value)

expectation value= 0.19708205121236266


To call the Hamiltonian we simply use

In [ ]:
print(NSMHamiltonian.hamiltonian)